# Введение

Данные интерактивные тетради основаны на языке Python.

Для выполнения кода выберите ячейку с кодом и нажмите `Ctrl + Enter`.

In [ ]:
from platform import python_version
print("Используемая версия Python:", python_version())

Ячейки подразумевают последовательное исполнение.

## Математический аппарат

В этих интерактивных тетрадях используется математический аппарат, основанный на парах вектор-кватернион.

Вектор (`Vector`) представлен тремя чиселами, кватернион (`Quaternion`) - четыремя.
Пара вектор-кватернион (`Transformation`) соостоит вектора и кватерниона и описывает последовательные перемещение и поворот.
$$ T = 
\begin{bmatrix}
    [v_x, v_y, v_z] \\
    [q_w, q_x, q_y, q_z]
\end{bmatrix}
$$

Математический аппарат расположен в файле [kinematics.py](../edit/kinematics.py)

### Vector

In [ ]:
from kinematics import Vector

Создание вектора требует трех чисел:

In [ ]:
v1 = Vector(1, 2, 3)
v2 = Vector(-2, 4, -3)

Вектора можно складывать поэлементно:

In [ ]:
v1 + v2

А также умножать на скаляр:

In [ ]:
2.5 * v1

Нулевой вектор создается через `Vector.zero()`:

In [ ]:
Vector.zero()

### Quaternion

In [ ]:
from kinematics import Quaternion
from numpy import pi

Кватернион создается из угла и оси поворота:

In [ ]:
q1 = Quaternion.from_angle_axis(0.5 * pi, Vector(0, 0, 1))
q2 = Quaternion.from_angle_axis(0.5 * pi, Vector(1, 0, 0))
print(q1)
print(q2)

Перемножение кватернионов соответствует последовательному приложению поворотов, в данном случае - повороту вокруг оси, проходящей через точку `(1, 1, 1)` на угол 120 градусов:

In [ ]:
q1 * q2

In [ ]:
Quaternion.from_angle_axis(2 / 3 * pi, Vector(1, 1, 1).normalized())

Поворот вектора сокращен до оператора `*`:

In [ ]:
Quaternion.from_angle_axis(pi / 2, Vector(0, 0, 1)) * Vector(1, 2, 3)

Кватернион нулевого поворота создается `Quaternion.identity()`:

In [ ]:
Quaternion.identity() * Vector(1, 2, 3)

### Transform

In [ ]:
from kinematics import Transform

Пара вектор-кватернион собирается из вектора и кватерниона:

In [ ]:
t1 = Transform(v1, q1)
t2 = Transform(v2, q2)

Пара состоит из смещения и поворота:

In [ ]:
t1.translation

In [ ]:
t1.rotation

Пара с нулевыми смещением и поворотом создается через `Transform.identity()`:

In [ ]:
Transform.identity()

Суммирование двух пар описывет последовательное применение смещения - поворота - смещения - поворота:

In [ ]:
t1 + t2

Суммирование пары и ветора описывает применение преобразования, записанного в паре к вектору:

In [ ]:
t1 + Vector(1, 0, 0)

## Графика

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
import graphics
%matplotlib notebook

Отрисовка систем координат производится через `graphics.axis`.
Преобразование цепочки в отдельные массивы точек `X, Y, Z` производится через `graphics.chain_to_points`.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlim([-3, 3]); ax.set_ylim([-3, 3]); ax.set_zlim([-3, 3]);
graphics.axis(ax, Transform.identity(), 3)
graphics.axis(ax, t1)
graphics.axis(ax, t1 + t2)
x, y, z = graphics.chain_to_points([Transform.identity(), t1, t1 + t2])
ax.plot(x, y, z)
fig.show()

## Анимация

Анимация будет сохраняться в переменную, например в `ani`, которую потом можно будет отобразить в виде видеоролика через `HTML(ani.to_jshtml())`.
Перед сохранением в виде ролика можно заранее повернуть сцену мышкой.
Обратите внимание что перерисовка каждого кадра требует работы ядра.
Для остановки нажмите кнопку выключения в правом верхнем углу трехмерной сцены.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlim([-1, 1]); ax.set_ylim([-1, 1]); ax.set_zlim([0, 2 * pi])
l, = ax.plot([], [], [])
t = np.arange(1, 2 * pi, 0.1)
frames = 100
def animate(i):
    offs = i / frames * 2 * pi
    z = t
    q = Quaternion.from_angle_axis(t + offs, Vector(0, 0, 1))
    v = q * Vector(1, 0, 0)
    x = v.x
    y = v.y
    l.set_data_3d(x, y, z)
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=frames,
    interval=100
)

Не забудьте выключить пересчет модели кнопкой в правом верхнем углу трехмерной сцены.

In [ ]:
HTML(ani.to_jshtml())

Полученый таким образом ролик можно сохранить в составе всей тетради и выкачать локальную копию через `File -> Download as -> Notebook (.ipynb)`.